In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import scipy.sparse as sp


import denseRSRG as df
import exactdiago as ex

In [3]:
def initialize_operators(m, l):
  """
  initialize_operators _summary_

  Parameters
  ----------
  m : _type_
      _description_
  l : _type_
      _description_

  Returns
  -------
  _type_
      _description_
  """
  s_x, _, s_z = df.pauli_matrices()
  
  A_L_0 = A_R_0 = np.eye(2**m)
  B_L_0 = np.kron(np.eye(2**(m - 1)), s_x)
  B_R_0 = np.kron(s_x, np.eye(2**(m - 1)))
  
  H_L_0 = H_R_0 = df.ising_hamiltonian(m,l) ## TO CHECK
  
  return A_L_0, B_L_0, A_R_0, B_R_0, H_L_0, H_R_0

# ===========================================================================================================

def compute_H_LR(H_L, H_R, AL, BL, AR, BR, l):
  s_x, _, s_z = df.pauli_matrices()
  H_L1 = np.kron(H_L, np.eye(2)) + np.kron(AL, l * s_z) + np.kron(BL, s_x)
  H_R1 = np.kron(np.eye(2), H_R) + np.kron(l * s_z, AR) + np.kron(s_x, BR)
  return H_L1, H_R1

# ===========================================================================================================

def update_operators(A_L, B_L, A_R, B_R, m): 
  s_x, _, _ = df.pauli_matrices()
  
  A_L_new = np.kron(np.eye(2), A_L)
  B_L_new = np.kron(s_x, B_L)
  
  A_R_new = np.kron(A_R, np.eye(2))
  B_R_new = np.kron(B_R, s_x)
  
  return A_L_new, B_L_new, A_R_new, B_R_new

# ===========================================================================================================

def compute_H_2m(H_L1, H_R1, m):
  s_x, _, _ = df.pauli_matrices()
  # Create the interaction term H_LR: Identity matrices on both sides with Pauli-X in the middle
  I_m = np.eye(2**(m))
  I_m1 = np.eye(H_R1.shape[1])

  H_int = np.kron(s_x, s_x)
  H_LR = np.kron(I_m, np.kron(H_int, I_m))  ## to check
  
  H_2m = np.kron(H_L1, I_m1) + np.kron(I_m1, H_R1) + H_LR
  
  return H_2m

# ===========================================================================================================
  
def rdm(psi, N, D, keep_indices):
  """
  rdm :
    Computes the reduced density matrix of a quantum state by tracing out the 
    degrees of freedom of the environment.

  Parameters
  ----------
  psi : np.ndarray
    Wavefunction of the quantum many-body system, represented as a complex vector of 
    size D^N.
  N : int
    Number of subsystems.
  D : int
    Dimension of each subsystem.
  keep_indices : list of int
    Indices of the sites to retain in the subsystem (all other sites are traced out).

  Returns
  -------
  rdm : np.ndarray
    Reduced density matrix for the subsystem specified by keep_indices, which is a 
    square matrix of size (D^len(keep_indices), D^len(keep_indices)).
  """
  # Check correct values for 'keep_indices'
  if not all(0 <= idx < N for idx in keep_indices):
    raise ValueError(f"'keep_indices' must be valid indices within range(n_sites), got {keep_indices}")
    
  # Compute subsystem and environment dimensions
  n_keep = len(keep_indices)
  subsystem_dim = D ** n_keep
  env_dim = D ** (N - n_keep)

  # Reshape the wavefunction into a sparse tensor (use csr_matrix for efficient sparse storage)
  psi_tensor = psi.reshape([D] * N)

  # Reorder the axes to group subsystem (first) and environment (second)
  all_indices = list(range(N))
  env_indices = [i for i in all_indices if i not in keep_indices]  # complement of keep_indices
  reordered_tensor = np.transpose(psi_tensor, axes=keep_indices + env_indices)

  # Partition into subsystem and environment (reshape back)
  psi_partitioned = reordered_tensor.reshape((subsystem_dim, env_dim))

  # Compute the reduced density matrix (use sparse matrix multiplication)
  rdm = psi_partitioned.dot(psi_partitioned.conj().T)

  return rdm


def get_reduced_density_matrix(psi, loc_dim, n_sites, keep_indices,
    print_rho=False):
    """
    Parameters
    ----------
    psi : ndarray
        state of the Quantum Many-Body system
    loc_dim : int
        local dimension of each single site of the QMB system
    n_sites : int
        total number of sites in the QMB system
    keep_indices (list of ints):
        Indices of the lattice sites to keep.
    print_rho : bool, optional
        If True, it prints the obtained reduced density matrix], by default False

    Returns
    -------
    ndarray
        Reduced density matrix
    """
    if not isinstance(psi, np.ndarray):
        raise TypeError(f'density_mat should be an ndarray, not a {type(psi)}')

    if not np.isscalar(loc_dim) and not isinstance(loc_dim, int):
        raise TypeError(f'loc_dim must be an SCALAR & INTEGER, not a {type(loc_dim)}')

    if not np.isscalar(n_sites) and not isinstance(n_sites, int):
        raise TypeError(f'n_sites must be an SCALAR & INTEGER, not a {type(n_sites)}')

    # Ensure psi is reshaped into a tensor with one leg per lattice site
    psi_tensor = psi.reshape(*[loc_dim for _ in range(int(n_sites))])
    # Determine the environmental indices
    all_indices = list(range(n_sites))
    env_indices = [i for i in all_indices if i not in keep_indices]
    new_order = keep_indices + env_indices
    # Rearrange the tensor to group subsystem and environment indices
    psi_tensor = np.transpose(psi_tensor, axes=new_order)
    print(f"Reordered psi_tensor shape: {psi_tensor.shape}")
    # Determine the dimensions of the subsystem and environment for the bipartition
    subsystem_dim = np.prod([loc_dim for i in keep_indices])
    env_dim = np.prod([loc_dim for i in env_indices])
    # Reshape the reordered tensor to separate subsystem from environment
    psi_partitioned = psi_tensor.reshape((subsystem_dim, env_dim))
    # Compute the reduced density matrix by tracing out the env-indices
    RDM = np.tensordot(psi_partitioned, np.conjugate(psi_partitioned), axes=([1], [1]))
    # Reshape rho to ensure it is a square matrix corresponding to the subsystem
    RDM = RDM.reshape((subsystem_dim, subsystem_dim))

    # PRINT RHO
    if print_rho:
        print('----------------------------------------------------')
        print(f'DENSITY MATRIX TRACING SITES ({str(env_indices)})')
        print('----------------------------------------------------')
        print(RDM)

    return RDM

# ===========================================================================================================

def projector(rho_L, k):
  
  if k > rho_L.shape[0]:
    raise ValueError(f"'k' must be <= the dimension of rho_L, got k={k} and dim={rho_L.shape[0]}")

  eigvals, eigvecs = np.linalg.eigh(rho_L)
  sorted_indices = np.argsort(eigvals)[::-1]  ##TO CHECK

  eigvals = eigvals[sorted_indices]
  eigvecs = eigvecs[:, sorted_indices]

  eigvals = eigvals[:k]
  eigvecs = eigvecs[:, :k]

  proj = eigvecs
  return proj

# ===========================================================================================================

def truncate_operators(P_L, P_R, A_L, B_L, A_R, B_R, H_L, H_R):
  P_L_dagger = P_L.conj().T
  P_R_dagger = P_R.conj().T
  
  A_L_trunc = P_L_dagger @ A_L @ P_L
  B_L_trunc = P_L_dagger @ B_L @ P_L
  A_R_trunc = P_R_dagger @ A_R @ P_R
  B_R_trunc = P_R_dagger @ B_R @ P_R
  
  H_L_trunc = P_L_dagger @ H_L @ P_L
  H_R_trunc = P_R_dagger @ H_R @ P_R

  return A_L_trunc, B_L_trunc, A_R_trunc, B_R_trunc, H_L_trunc, H_R_trunc

# ===========================================================================================================

def dmrg(l, m_max, threshold=1e-6, max_iter=100):
  """
  Infinite DMRG function for a 1D quantum system.

  Parameters
  ----------
  l : float
      Coupling parameter (e.g., transverse field strength).
  m_max : int
      Maximum number of states to retain during truncation.
  convergence_threshold : float, optional
      Convergence criterion for the energy difference between iterations.
  max_iterations : int, optional
      Maximum number of iterations allowed.

  Returns
  -------
  E_ground : float
      Ground state energy per site in the thermodynamic limit.
  """
  # Initialize operators and Hamiltonians
  m = 1  # Initial single-site system
  A_L, B_L, A_R, B_R, H_L, H_R = initialize_operators(m, l)

  # Track energy and iteration
  actual_dim = 2*m
  prev_energy_density = np.inf
  
  for iteration in range(max_iter):
    # Step 1: Enlarge Hamiltonians
    H_L1, H_R1 = compute_H_LR(H_L, H_R, A_L, B_L, A_R, B_R, l)
    
    # Step 2: Combine into full system Hamiltonian
    H_2m = compute_H_2m(H_L1, H_R1, m)

    E, psi = np.linalg.eigh(H_2m)
    sorted_indices = np.argsort(E)  

    E = E[sorted_indices]
    psi = psi[:, sorted_indices]

    E_ground = E[0]
    psi_ground = psi[:, 0]

    # Step 4: Compute reduced density matrix
    N = int(np.log2(H_2m.shape[0]))
    D = 2  # Local Hilbert space dimension
    keep_indices_left = list(range(0, N // 2))  # Keep left block sites
    # rho_L = rdm(psi_ground, N, D, keep_indices)
    rho_L = get_reduced_density_matrix(psi_ground, D, N, keep_indices_left, print_rho=False)
    
    keep_indices_right = list(range(N // 2, N))  # Keep left block sites
    # rho_L = rdm(psi_ground, N, D, keep_indices)
    rho_R = get_reduced_density_matrix(psi_ground, D, N, keep_indices_right, print_rho=False)


    # Step 5: Construct the projector
    k = min(2 ** m_max, rho_L.shape[0] - 1)  # Ensure k does not exceed the dimension
    P_L= projector(rho_L, k) 
    P_R = projector(rho_R, k)    
    
    # Step 6: Truncate operators and Hamiltonians
    A_L, B_L, A_R, B_R = update_operators(A_L, B_L, A_R, B_R, m)
    A_L, B_L, A_R, B_R, H_L, H_R = truncate_operators(P_L, P_R, A_L, B_L, A_R, B_R, H_L1, H_R1)

    # Step 7: Check convergence
    actual_dim = actual_dim + 2
    current_energy_density = E_ground / actual_dim
    delta = abs(current_energy_density - prev_energy_density)

    if delta < threshold:
      print(f"Converged after {iteration + 1} iterations.")
      break

    # Update for the next iteration
    prev_energy_density = current_energy_density
      
    if iteration % 10 == 0:
      print(f"Starting iteration {iteration} ...")
    
  print(f"Reached N = {actual_dim} with precision: delta = {delta}")
  return current_energy_density, E_ground, psi_ground

In [4]:
l_test = 1.0  # Example coupling parameter (e.g., transverse field strength)
m_max_test = 1  # Maximum number of states to retain during truncation
convergence_threshold_test = 1e-4  # Convergence threshold
max_iterations_test = 50  # Maximum number of iterations

energy_density, E_ground, psi_ground = dmrg(l_test, m_max_test, convergence_threshold_test, max_iterations_test)

# Print the result
print(f"Ground state energy per site: {energy_density}")

Reordered psi_tensor shape: (2, 2, 2, 2)
Reordered psi_tensor shape: (2, 2, 2, 2)


UnboundLocalError: cannot access local variable 'prev_energy_density' where it is not associated with a value